---
<center>

  # **Tarea 06**

**Realizado Por:**

   Samuel Huertas Rojas

---
</center>

# 1. 
Compruebe el teorema del virial para el átomo de Hidrógeno a partir de los elementos de matriz
de los operadores de energía cinética y potencial.

El teorema del virial nos dice que:

<center>

$
\braket{T} = - \dfrac{1}{2} \braket{U}
$

</center>

Para comprobar el teorema del virial, vamos a utilizar la representación matricial de cada unos de los operadores de energía cinética y potencial, los cuales se definen como:

* Energía cinética:

<center>

$
\hat{T} = - \dfrac{\hbar^2}{2 \mu} \nabla^2 = - \dfrac{\hbar^2}{2 \mu} \left(\dfrac{d^2}{dr^2} + \dfrac{2}{r} \dfrac{d}{dr} - \dfrac{l(l+1)}{r^2} \right)
$

</center>

* Energía potencial:

<center>

$
\hat{U} = - \dfrac{Z}{r}
$

</center>

In [ ]:
import sympy as sp
from sympy.physics.hydrogen import R_nl
from sympy import symbols, simplify, diff

# Constantes (unidades atómicas)
a0 = 1.0  # Radio de Bohr en unidades atómicas
e = 1.0  # Carga del electrón en unidades atómicas
hbar = 1.0  # Constante de Planck reducida en unidades atómicas
me = 1.0  # Masa del electrón en unidades atómicas
mu = 1  # Masa reducid= mM/(m + M)
Z = 1  # Número átomico, número de protones en el núcleo de un átomo

# Definición de los simbolos a utilizar en las coordenadas esfericas
r, theta, phi = symbols("r theta phi", real=True)


# Definición de la función de onda del átomo de hidrogeno
def funcion_onda(n, l, Z=1):  # noqa: E741
    """
    Calcula la función radial R_nl(r) del átomo de hidrógeno.
    
    Parámetros:
    -----------
    n : int
        Número cuántico principal (n >= 1)
    l : int
        Número cuántico orbital (0 <= l < n)
    Z : int
        Número atómico (Z=1 para hidrógeno)
    usar_sympy : bool
        Si True, usa la función incorporada de SymPy
        Si False, implementa la fórmula manualmente
    
    Retorna:
    --------
    R_nl : expresión simbólica de SymPy
        La función radial en términos de r
    """
    
    if l >= n:
        raise ValueError(f"l debe ser menor que n. Recibido: n={n}, l={l}")

    R = R_nl(n, l, r, Z)
    return simplify(R)
    


# Definición del operador de la energía cinética en coordenadas esfericas
def energia_cinetica(f, l):  # noqa: E741
    E = (
        -(hbar**2) / (2 * mu) * ( 
            diff(f, r, 2)
            + 2 / r * diff(f, r)
            - (l * (l + 1)) / r**2 
        )
    )
    return E


# Definición del operedaro de la energía potencial
def energia_potencial(f):
    E = -Z / r * f
    return E

# Obtención de los elementos de matriz de los operadores
# Operador energía cinética
def elemento_cinetico(n, l):  # noqa: E741
    Psi = funcion_onda(n, l)  # noqa: E741
    Psi_cinetico = energia_cinetica(Psi, l)
    Psi_conjugado = sp.conjugate(Psi)

    # Cálculo de la energía esperada
    integrando = Psi_conjugado * Psi_cinetico * r**2 * sp.sin(theta)

    return sp.integrate(
        sp.integrate(sp.integrate(integrando, (r, 0, sp.oo)), (theta, 0, sp.pi)),
        (phi, 0, 2 * sp.pi),
    )

    

# Operador energía potencial
def elemento_potencial(n, l):  # noqa: E741
    Psi = funcion_onda(n, l)  # noqa: E741
    elem = energia_potencial(Psi)

    # Cálculo de la energía esperada
    integrando = sp.conjugate(Psi) * elem * r**2 * sp.sin(theta)

    return sp.integrate(
        sp.integrate(sp.integrate(integrando, (r, 0, sp.oo)), (theta, 0, sp.pi)),
        (phi, 0, 2 * sp.pi),
    )

    

# Verificación del teorema del virial
print("=" * 70)
print("VERIFICACIÓN DEL TEOREMA DEL VIRIAL EN EL ÁTOMO DE HIDRÓGENO")
print("=" * 70)
print("  • Teorema del Virial: ⟨T⟩ = -½⟨V⟩")
print("\n" + "=" * 70)

# Estados a verificar
estados = [
    (1, 0, 0),  # 1s
    (2, 0, 0),  # 2s
    (2, 1, 0),  # 2p
    (3, 0, 0),  # 3s
    (3, 1, 0),  # 3p
    (3, 2, 0),  # 3d
]

resultados = []

for n, l, m in estados:  # noqa: E741
    T_calc = elemento_cinetico(n, l)
    U_calc = elemento_potencial(n, l)
    
    # Verificación del teorema del virial
    virial_ratio = T_calc / (-0.5 * U_calc)
    
    resultados.append({
        'state': f"{n}{'spdfgh'[l]}",
        'n': n, 'l': l,
        'T': T_calc/2,
        'V': U_calc,
        'virial_ratio': virial_ratio
    })
    
    print(f"\nEstado: {n}{'spdfgh'[l]} (n={n}, l={l}, m={m})")
    print(f"  ⟨T⟩ = {T_calc:12.8f} a.u.")
    print(f" -1/2 ⟨V⟩ = {U_calc:12.8f} a.u.")
    print("\n  Verificación del Teorema del Virial:")
    print(f"    ⟨T⟩ / (-½⟨V⟩) = {virial_ratio:12.8f} (debe ser ≈ 1)")


VERIFICACIÓN DEL TEOREMA DEL VIRIAL EN EL ÁTOMO DE HIDRÓGENO
  • Teorema del Virial: ⟨T⟩ = -½⟨V⟩


Estado: 1s (n=1, l=0, m=0)
  ⟨T⟩ =   0.00000000 a.u.


TypeError: unsupported format string passed to NaN.__format__

# 2. 
Encuentre la corrección relativista a la energía cinética calculando el elemento de matriz del
operador energía cinética al cuadrado.

# 3.
Escriba las expresiones analíticas de las correcciones relativistas y compruebe que se obtiene lo
mismo que en el punto anterior.

# 4.
Diagonalice la matriz de espín-órbita para el estado $n=3$, $l=1$ y compruebe que da lo mismo que
la expresión analítica.

# 5. 
Compruebe que los autoestados de la matriz anterior son las autofunciones de $J$ y $Jz$
comparándolos con los coeficientes de Clebsch Gordan.